In [1]:
import numpy as np
import pandas as pd
from Function_readLabChartMat import get_pages_data, backRMS, autoEventDetection, manualEventDetect, avgVals, get_page_info, importPagesLabMat,mark_pages

In [2]:
filename = r"R:\Prosjekter\ELECTROPHYSIOLOGY_PD_1000214\HealthyControl\EHC03\EHC03_ShortTrial_15trials_30042024.mat"
preTrigger = 50 #time in msec
postTrigger = 100 #time in msec
samp_rate = 4000
pre_time_ms = 50 
post_time_ms = 100
avgFact = 15

In [3]:
(data_by_block, com) = importPagesLabMat(filename)

In [4]:
print(len(data_by_block[0]))

8


In [5]:
print(len(com))

225


In [6]:
marked_pages = mark_pages(filename,samp_rate, pre_time_ms, post_time_ms)

In [7]:
fileNamePath = filename
import numpy as np
import scipy.io

# using scipy to load matlab file. Note data is imported as arrays ndarray
mat = scipy.io.loadmat(fileNamePath)

# extracting data
allData = mat['data']
allData = allData.astype(np.double)
com = mat['com']

# extracting dataStart and dataEnd indices + defining the number of blocks
dataStart = mat['datastart']
dataStart = dataStart.astype(np.int32)
dataEnd = mat['dataend']
dataEnd = dataEnd.astype(np.int32)
nosBlocks = dataStart.shape[1]

In [8]:
(data_by_block, com) = importPagesLabMat(filename)
nos_blocks = len(data_by_block)

In [9]:
pages_by_block = get_page_info(com, samp_rate, pre_time_ms, post_time_ms)

# creating marked_pages as a copy of data_by_block for further modifications
marked_pages = data_by_block.copy()
unique_blocks = np.unique(com[:, 1])
block_no = 0
pg_no = 0  # this is page within a block
given_page_no = 1  # this is labChart page number - note this so that page number doesn't go back to 1 for each new block

In [49]:
import numpy as np 

# Getting data sorted by blocks 
marked_pages = mark_pages(filename,samp_rate, pre_time_ms, post_time_ms)

club_data = np.hstack(marked_pages)
    
# Identify page numbers and remove zeros
page_nos = np.unique(club_data[-1])
page_nos = page_nos[page_nos > 0] # note: selecting data that belongs to pages

#starting a loop to create pages with all channels information in that page
pages = []
for pg_no in page_nos:
    ind_pg = np.where(club_data[-1] == pg_no)[0] #finding the indices of data belonging to current page 
    pg_val = club_data[:,ind_pg] #double check if there is a need to switch columns and rows
    pages.append(pg_val) #maybe first convert to list before appending?

# OPTIONAL: averaging the pages if asked by the user
if avgFact != 0:
    a1 = 0
    diff = avgFact
    noLots = int(len(pages) / avgFact) # this gives number of resulting pages after averaging
    chNos = len(pages[0]) # this gives number of channels in the first page - should be common to all pages
    new_lotArray = []
    for lotNo in range(1,noLots+1): #double check +1 correction
        st_ind = a1 + (lotNo - 1)*diff #using arithmetic mean
        end_ind = st_ind + diff 
        useData = pages[st_ind:end_ind]
        mean_ChArray = []
        for chNo in range(0, chNos):
            pg_array4mean = []
            for pgNo in range(0, len(useData)):
                ind_Chnl = useData[pgNo][chNo]
                pg_array4mean.append(ind_Chnl)
            mean_pgArray = np.mean(pg_array4mean,axis = 0)
            mean_ChArray.append(mean_pgArray)
        new_lotArray.append(mean_ChArray)
    pages = new_lotArray

IndexError: list index out of range